# 🌐 LangGraph 기반 EXA 웹 검색 에이전트 실습

이번 실습에서는 **EXA Search API**를 활용한 웹 검색 에이전트를 **3가지 방식**으로 구현해보며, 각 방식의 특징과 장단점을 비교합니다.

> 📢 **EXA Search란?**
> 
> EXA는 AI 애플리케이션을 위해 설계된 **차세대 웹 검색 API**입니다.
> - **신경망 기반 검색**: 키워드가 아닌 의미 기반 검색
> - **실시간 웹 데이터**: 최신 정보 검색 가능
> - **고품질 결과**: SEO 스팸을 필터링한 깨끗한 결과

### 🎯 학습 목표
1. LangChain의 `create_agent`로 간단한 에이전트 구현
2. LangGraph의 StateGraph로 세밀한 Tool Call Routing 구현
4. 각 방식의 장단점 비교 및 선택 기준 이해


## 📋 목차

### Part A: LangChain create_agent (기본)
1. [환경 설정](#1-환경-설정)
2. [EXA Search Tool 생성](#2-exa-search-tool-생성)
3. [LangChain create_agent](#3-langchain-create_agent)

### Part B: LangGraph Tool Call Routing (심화)
4. [LangGraph 상태 정의](#4-langgraph-상태-정의)
5. [그래프 노드 및 엣지 구현](#5-그래프-노드-및-엣지-구현)
6. [Tool Call Routing 에이전트 실행](#6-tool-call-routing-에이전트-실행)


---
# 🔧 Part A: LangChain create_agent (기본)

## 1. 환경 설정

먼저 필요한 패키지들을 설치하고 환경을 구성합니다.


In [ ]:
# 필요한 패키지 설치
# %pip install -qU langchain langchain-openai langgraph langchain-exa exa-py python-dotenv
%pip install -qU langchain-exa exa-py


In [ ]:
# 환경 변수 설정
import os
from dotenv import load_dotenv

load_dotenv()

# API 키 설정
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# EXA API 키 (https://exa.ai 에서 발급)
if not os.getenv("EXA_API_KEY"):
    os.environ["EXA_API_KEY"] = "your-exa-api-key"

print("✅ 환경 설정 완료!")


---
## 2. EXA Search Tool 생성

EXA의 검색 기능을 LangChain Tool로 래핑합니다.


In [ ]:
from langchain_core.tools import tool
from exa_py import Exa

# EXA 클라이언트 초기화
exa_client = Exa(api_key=os.environ["EXA_API_KEY"])

@tool
def exa_web_search(query: str) -> str:
    """
    EXA를 사용하여 웹에서 최신 정보를 검색합니다.
    최신 뉴스, 기술 트렌드, 연구 논문 등을 검색할 때 사용하세요.
    
    Args:
        query: 검색할 질문이나 키워드
    """ 
    results = exa_client.search_and_contents(
        query=query,
        num_results=5,
        text={"max_characters": 1000},
        highlights={"num_sentences": 3}
    )
    
    output = []
    for result in results.results:
        output.append(f"**Title**: {result.title}")
        output.append(f"**URL**: {result.url}")
        if result.highlights:
            output.append(f"**Highlights**: {' '.join(result.highlights)}")
        output.append("---")
    
    return "\n".join(output)

print("✅ EXA Search Tool 생성 완료!")


In [ ]:
# EXA 검색 테스트
test_result = exa_web_search.invoke({"query": "LangGraph agent framework"})
print("🔍 EXA 검색 테스트:\n")
print(test_result[:1000] + "..." if len(test_result) > 1000 else test_result)


---
## 3. LangChain create_agent

LangChain의 **`create_agent`**는 ReAct(Reasoning + Acting) 패턴을 구현한 에이전트를 생성합니다.



agent = create_agent(llm, tools, prompt)
```


In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

# 도구 리스트
tools = [exa_web_search]

# 에이전트 생성
langchain_agent = create_agent(llm, tools)

print("✅ LangChain ReAct 에이전트 생성 완료!")


In [ ]:
from datetime import datetime
today = datetime.now().strftime("%Y년 %m월 %d일")

# LangChain 에이전트 테스트
test_question = "AI 에이전트 프레임워크 트렌드를 알려줘"

print(f"💬 질문: {test_question}")
print("=" * 50)


for chunk in langchain_agent.stream(  
    {"messages": [
        {"role": "system", "content": f"You are a helpful AI assistant with web search capabilities. Today is {today}"},
        {"role": "user", "content": test_question}]},
    stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

### 💡 LangChain create_agent 정리

**장점:**
- 빠른 프로토타이핑에 적합

**한계:**
- 복잡한 워크플로우 구현 어려움
- 조건부 분기, 루프 등 세밀한 제어 제한
- 상태 관리가 명시적이지 않음

👉 **더 복잡한 에이전트가 필요하다면? → LangGraph로!**


---
# 🔄 Part B: LangGraph Tool Call Routing (심화)

## 4. LangGraph 상태 정의

LangGraph는 **명시적인 상태 관리**와 **그래프 기반 워크플로우**를 제공합니다.

### 그래프 구조:
```
START → call_model → [tool_calls 있음?]
                       ├─ Yes → tools → call_model
                       └─ No → END
```

### 핵심 개념:
- **State**: 그래프 전체에서 공유되는 상태
- **Node**: 상태를 변환하는 함수
- **Edge**: 노드 간 연결 (조건부 가능)
- **ToolNode**: 도구 실행을 담당하는 prebuilt 노드


In [ ]:
from typing import Annotated, Literal
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage

# MessagesState: 메시지 리스트를 자동으로 관리하는 내장 상태
# messages 필드가 자동으로 누적됨

print("📊 MessagesState 구조:")
print("  - messages: Annotated[list, add_messages]")
print("  - 메시지가 자동으로 리스트에 추가됨")


---
## 5. 그래프 노드 및 엣지 구현

각 노드와 조건부 라우팅 로직을 구현합니다.


In [ ]:
from datetime import datetime
today = datetime.now().strftime("%Y년 %m월 %d일")

# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

# 1. 모델 호출 노드
def call_model(state: MessagesState):
    """LLM을 호출하여 응답 또는 도구 호출 결정"""
    print("---CALL MODEL---")
    
    # 시스템 프롬프트
    system_message = {
        "role": "system",
        "content": f"""You are a helpful AI assistant with web search capabilities. Today is {today}. 
Use exa_web_search for questions requiring current information with citations. Please always respond in Korean."""
    }
    
    messages = [system_message] + state["messages"]
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}

# 2. 조건부 라우팅 함수
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """도구 호출 필요 여부 판단"""
    last_message = state["messages"][-1]
    
    # tool_calls가 있으면 도구 실행
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("---ROUTING TO TOOLS---")
        return "tools"
    
    # 없으면 종료
    print("---ROUTING TO END---")
    return "__end__"

# 3. ToolNode 생성 (도구 실행 담당)
tool_node = ToolNode(tools)

print("✅ 노드 및 라우팅 함수 정의 완료!")


In [ ]:
# 그래프 조립
workflow = StateGraph(MessagesState)

# 노드 추가
workflow.add_node("call_model", call_model)
workflow.add_node("tools", tool_node)

# 엣지 연결
workflow.add_edge(START, "call_model")

# 조건부 엣지: call_model 후 도구 호출 여부에 따라 분기
workflow.add_conditional_edges(
    "call_model",
    should_continue,
    {
        "tools": "tools",
        "__end__": END
    }
)

# 도구 실행 후 다시 모델 호출
workflow.add_edge("tools", "call_model")

# 그래프 컴파일
langgraph_agent = workflow.compile()

print("✅ LangGraph Tool Call Routing 에이전트 컴파일 완료!")


In [ ]:
# 그래프 시각화
from IPython.display import Image, display

try:
    display(Image(langgraph_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 오류: {e}")
    print("그래프 구조: START → call_model → [tools/END] → call_model → END")


---
## 6. Tool Call Routing 에이전트 실행

구축한 LangGraph 에이전트를 테스트합니다.


In [ ]:
# 테스트 1: 일반 대화 (도구 호출 없음)
test1 = "안녕하세요!"

print(f"💬 질문: {test1}")
print("=" * 50)

result = langgraph_agent.invoke({"messages": [{"role": "user", "content": test1}]})
print(f"\n🤖 답변:\n{result['messages'][-1].content}")


In [ ]:
# 테스트 2: 웹 검색 필요한 질문
test2 = "LangGraph와 CrewAI의 차이점을 알려줘"

print(f"💬 질문: {test2}")
print("=" * 50)

result = langgraph_agent.invoke({"messages": [{"role": "user", "content": test2}]})
print(f"\n🤖 답변:\n{result['messages'][-1].content}")


In [ ]:
# 스트리밍으로 실행 과정 확인
test3 = "OpenAI의 최신 모델 소식을 알려줘"

print(f"💬 질문: {test3}")
print("=" * 50)
print("\n🔄 실행 과정:")

for chunk in langgraph_agent.stream({"messages": [{"role": "user", "content": test3}]}):
    for node_name, value in chunk.items():
        print(f"\n📍 [{node_name}] 노드 실행")
        
        if "messages" in value and value["messages"]:
            last_msg = value["messages"][-1]
            
            if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                for tc in last_msg.tool_calls:
                    print(f"   🔧 Tool: {tc['name']}")
            elif hasattr(last_msg, 'content') and last_msg.content:
                content = str(last_msg.content)
                if node_name == "call_model" and not (hasattr(last_msg, 'tool_calls') and last_msg.tool_calls):
                    print(f"\n🤖 최종 답변:\n{content}")


### 💡 LangGraph Tool Call Routing 정리

**장점:**
- **완전한 제어**: 모든 노드와 엣지를 직접 정의
- **상태 관리**: 명시적인 상태 추적 및 디버깅
- **확장성**: 복잡한 워크플로우, 조건부 분기, 루프 구현 가능
- **시각화**: 그래프 구조를 시각적으로 확인

**한계:**
- 더 많은 코드 작성 필요
- 간단한 에이전트에는 과도할 수 있음

👉 **더 간결하게 만들고 싶다면? → LangGraph prebuilt create_agent!**


## 🎯 Streaming 실습

LangGraph는 실시간 업데이트를 제공하는 스트리밍 시스템을 구현합니다.

**주요 Stream Modes:**
- `values`: 각 단계 후 전체 상태 스트리밍
- `updates`: 각 단계 후 상태 업데이트만 스트리밍
- `messages`: LLM 토큰 스트리밍
- `custom`: 사용자 정의 데이터 스트리밍
- `debug`: 상세한 디버깅 정보 스트리밍

In [ ]:
# messages 모드: LLM 토큰 실시간 스트리밍
print("=" * 60)
print("📊 Stream Mode: messages (LLM 토큰 스트리밍)")
print("=" * 60)

test_query = "OpenAI의 최신 모델 소식을 알려줘"

print(f"\n💬 질문: {test_query}\n")
print("🤖 답변: ", end="", flush=True)

for message_chunk, metadata in langgraph_agent.stream(
    {"messages": [{"role": "user", "content": test_query}]},
    stream_mode="messages"
):
    # LLM 토큰만 출력 (tool call은 제외)
    if message_chunk.content:
        print(message_chunk.content, end="", flush=True)

print("\n")